<h1 style="text-align: center;">Tennis Analyzer ML Model V2</h1>
<h3 style="text-align: center;">Dan Warnick</h3>

<p>To start we will begin by selecting the data points we want to analyze with existing known results. For each data entry we will have two players each with the following data entries.</p>
<table style="font-size: .8em;">
    <tr>
        <th>Player Name</th>
    </tr>
</table>
<p>Along with match facts like Clay/Hard/Grass Court or Indoor/Outdoor. In the future may want to add weather and adjust certain parameters for more accuracy and more data points to train from, however for now this seems a good start.</p>

<h2>1.) Collect Data Efficiently</h2>

In [1]:
import django
from django.http import HttpResponse
from django.template import loader
import os
from django.db import models
import torch
from math import ceil, floor
import math
import numpy as np
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils as utils
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from asgiref.sync import sync_to_async # type: ignore
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
import copy
import joblib

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'breakpoint.settings')
# django.setup()

# from render.models import *

In [2]:
start = '20120101'
end = '20231231'
match_type = 'm'

In [3]:
start_date = datetime.strptime(start, '%Y%m%d').date()
end_date = datetime.strptime(end, '%Y%m%d').date()

# if match_type == 'm':   
#     typer = MensTennisMatch
#     insert_db = MensTennisMatchStats
# else:
#     typer = WomensTennisMatch
#     insert_db = WomensTennisMatchStats

# query = insert_db.objects.filter(
#         tourney_date__range=(start_date, end_date)
#     ).order_by('tourney_date')
    
# games = await sync_to_async(list)(query.all().values())

RD_CUTOFF = 125

df = pd.read_csv('../../testcsvs/StephNRP25.csv')

print(df.columns)
df['tourney_date'] = pd.to_datetime(df['tourney_date']).dt.date
df = df[(df['tourney_date'] >= start_date) & (df['tourney_date'] <= end_date)]
# df = df[(df['a_glicko_rd'] <= RD_CUTOFF) & (df['b_glicko_rd'] <= RD_CUTOFF)]

df = df.drop(['tourney_id', 'tourney_name', 'match_num', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug', 'sets', 'games', 'tiebreaks'], axis=1)
# print(df.duplicated().sum())

print(len(df))

one_hot_encoder = OneHotEncoder(sparse_output=False)

# One-Hot Encode the 'category_text' column
category_encoded = one_hot_encoder.fit_transform(df[['surface']])

# Convert to DataFrame
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names_out(['surface']))

# Concatenate the one-hot encoded columns back to the original DataFrame
df = pd.concat([df.reset_index(drop=True), category_encoded_df], axis=1)

# Drop the original 'category_text' column
df.drop('surface', axis=1, inplace=True)
# print(df.duplicated().sum())

print(len(df))

Index(['tourney_id', 'tourney_name', 'tourney_date', 'surface', 'best_of',
       'match_num', 'tourney_level', 'tourney_round', 'a_player_id',
       'a_player_name',
       ...
       'a_surface_return_second_won_steph_rating',
       'b_surface_second_won_steph_rating',
       'a_surface_return_second_won_steph_rd', 'b_surface_second_won_steph_rd',
       'sets', 'games', 'tiebreaks', 'a_odds', 'b_odds', 'a_b_win'],
      dtype='object', length=174)


KeyError: 'a_glicko_rd'

In [15]:
df.head()

,best_of,tourney_level,tourney_round,a_player_age,a_player_hand,a_player_ht,a_player_rank,a_player_rank_points,b_player_age,b_player_hand,...,a_surface_return_second_won_glicko_rating,b_surface_second_won_glicko_rating,a_surface_return_second_won_glicko_rd,b_surface_second_won_glicko_rd,a_odds,b_odds,a_b_win,surface_Clay,surface_Grass,surface_Hard
0,3.0,2.0,0.8,28.0,0.0,185.0,74.0,671.0,23.0,1.0,...,1511.785013,1528.178280,65.035496,61.424189,3.59,1.28,0.0,0.0,0.0,1.0
1,3.0,2.0,0.8,28.0,1.0,188.0,65.0,755.0,25.0,1.0,...,1522.769999,1495.331465,62.134827,70.640554,NaN,NaN,1.0,0.0,0.0,1.0
2,3.0,2.0,0.8,23.0,1.0,183.0,89.0,612.0,22.0,1.0,...,1492.043641,1527.915304,78.973379,83.154575,2.29,1.59,0.0,0.0,0.0,1.0
3,3.0,2.0,0.8,24.0,1.0,178.0,48.0,915.0,26.0,1.0,...,1510.305024,1508.599843,63.986136,65.213573,2.40,1.54,0.0,0.0,0.0,1.0
4,3.0,2.0,0.8,25.0,1.0,193.0,22.0,1655.0,26.0,1.0,...,1510.529509,1475.691709,61.507367,80.902188,1.19,4.44,1.0,0.0,0.0,1.0


In [ ]:
import sqlite3

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


df = df.dropna()

margin = df['a_b_win']
df = df.drop(columns=['a_b_win', 'a_odds', 'b_odds', 'tourney_level'])

df = df.values

df = df.astype(float)
acc_results = []
for x in tqdm(range(200)):
    x_train, x_test, y_train, y_test = train_test_split(df, margin, test_size=.1)

    train = xgb.DMatrix(x_train, label=y_train)
    test = xgb.DMatrix(x_test, label=y_test)

    param = {
        'max_depth': 3,
        'eta': 0.01,
        'objective': 'multi:softprob',
        'num_class': 2
    }
    epochs = 750

    model = xgb.train(param, train, epochs)
    predictions = model.predict(test)
    y = []

    for z in predictions:
        y.append(np.argmax(z))

    acc = round(accuracy_score(y_test, y) * 100, 1)
    print(f"{acc}%")
    acc_results.append(acc)
    # only save results if they are the best so far
    if acc == max(acc_results):
        model.save_model('XGBoost_{}%_ML-4.json'.format(acc))

  0%|          | 1/300 [00:02<11:58,  2.40s/it]

100.0%


  1%|          | 2/300 [00:04<11:18,  2.28s/it]

100.0%


  1%|          | 3/300 [00:08<14:36,  2.95s/it]

...

100.0%


100%|█████████▉| 299/300 [12:06<00:02,  2.05s/it]

100.0%


100%|██████████| 300/300 [12:08<00:00,  2.43s/it]

100.0%
